In [1]:
import pandas as pd

In [ ]:
plants = pd.read_excel('egrid2019_data.xlsx', 'PLNT'+'19', skiprows=[0])
df = pd.read_csv('egrid_2016_plant_easiur.csv')

# Fill in missing data

In [118]:
plants.loc[(plants['NERC']=='WECC')&(plants['LAT'].isna())]

,SEQPLT19,YEAR,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,SECTOR,...,PLWIPR,PLSOPR,PLGTPR,PLOFPR,PLOPPR,PLTNPR,PLTRPR,PLTHPR,PLCYPR,PLCNPR
7964,7965,2019,NM,Eddy County Generating Station,55252,NaN,-9999,Public Service Co of NM,15473,Electric Utility,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10868,10869,2019,TX,Turbine,7732,El Paso Electric Co,5701,El Paso Electric Co,5701,Electric Utility,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11481,11482,2019,WA,SPI - Everett,56281,NaN,-9999,Sierra Pacific Industries Inc,17164,Industrial Non-CHP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
# Fill in for three plants with missing location data, based on google maps investigation

plants.loc[7964, 'LON'] = -104.169391; plants.loc[7964, 'LAT'] = 32.356716 # best guess at location, within eddy
plants.loc[10868, 'LON'] = -106.432; plants.loc[10868, 'LAT'] = 31.9836 # location of nearby other plant
plants.loc[11481, 'LAT'] = 47.923575; plants.loc[11481, 'LON'] = -122.102489 # industrial location in Snohomish



In [131]:
new_df = plants.loc[:, ['SEQPLT19', 'PSTATABB', 'PNAME', 'ORISPL', 'OPRNAME', 'OPRCODE',
       'UTLSRVNM', 'UTLSRVID', 'BANAME', 'BACODE', 'NERC', 'SUBRGN', 'SRNAME',
       'ISORTO', 'FIPSST', 'FIPSCNTY', 'CNTYNAME', 'LAT', 'LON']].copy(deep=True)

In [132]:
new_df = new_df.loc[(~(new_df['PSTATABB'].isin(['AK', 'PR', 'HI'])))&(~(new_df['LAT'].isna()))].reset_index(drop=True).copy(deep=True)

# Send this file to https://barney.ce.cmu.edu/~jinhyok/easiur/online/

In [134]:
new_df.loc[:, ['LON', 'LAT']].to_csv('easiur_input_file.csv', header=None, index=None)

In [136]:
new_df.loc[(new_df['LAT'].isna())]

,SEQPLT19,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,BANAME,BACODE,NERC,SUBRGN,SRNAME,ISORTO,FIPSST,FIPSCNTY,CNTYNAME,LAT,LON


# Receive this file back

In [137]:
output = pd.read_csv('easiur_easiurinputfile.csv')
output.head()

,Longitude,Latitude,x,y,PM25 Annual Ground,PM25 Winter Ground,PM25 Spring Ground,PM25 Summer Ground,PM25 Fall Ground,SO2 Annual Ground,...,NOX Annual 300m,NOX Winter 300m,NOX Spring 300m,NOX Summer 300m,NOX Fall 300m,NH3 Annual 300m,NH3 Winter 300m,NH3 Spring 300m,NH3 Summer 300m,NH3 Fall 300m
0,-86.779866,33.582793,103,40,215304.728918,245017.615571,203972.457312,188898.160133,222976.443192,24581.526295,...,3997.135985,8402.340713,4770.644606,1516.761629,1193.553982,69164.427372,80304.724163,65320.617657,69484.988913,61463.141248
1,-85.310833,32.748611,107,38,122786.497026,149360.745246,111240.493722,96995.909112,133078.008247,22378.652632,...,3217.535556,7531.222643,3007.045705,1014.927945,1213.618474,46834.162986,56360.503260,39562.879637,45700.799126,45490.147238
2,-87.488900,31.582500,102,34,78879.537870,98244.951378,63055.611800,65868.356232,88016.400863,20563.833895,...,2563.447203,6322.469309,2134.010860,948.677790,757.601053,33813.845499,43984.759154,22839.883428,29595.491507,38634.389520
3,-87.488900,31.582500,102,34,78879.537870,98244.951378,63055.611800,65868.356232,88016.400863,20563.833895,...,2563.447203,6322.469309,2134.010860,948.677790,757.601053,33813.845499,43984.759154,22839.883428,29595.491507,38634.389520
4,-86.223889,34.363389,104,43,141280.719433,165153.985216,122378.742710,117771.516915,159559.217278,24898.816004,...,4538.242839,10735.446174,4240.916332,1516.574017,1519.786803,50409.835550,67215.719470,34847.120853,35995.838231,63070.267832


# Save in correct format for model

In [146]:
new_df = pd.concat((new_df, output.loc[:, output.columns[24:44]]), axis=1)

In [150]:
new_df.to_csv('egrid_2019_plant_easiur.csv', index=None)